<a href="https://colab.research.google.com/github/shizoda/education/blob/main/machine_learning/basics/neural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 機械学習体験プログラム

基本的な分類問題を用いて機械学習を体験しつつ、ニューラルネットワークの仕組みや使い方を理解しましょう。

<a title="Dake, Mysid, CC BY 1.0 &lt;https://creativecommons.org/licenses/by/1.0&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Neural_network.svg"><img width="512" alt="Neural network" src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/3d/Neural_network.svg/512px-Neural_network.svg.png?20210102213114"></a>

## 必要なライブラリ

以下のライブラリをインポートします：
- `numpy`: 数値計算を効率的に行うためのライブラリ
- `scikit-learn`: 機械学習のためのライブラリ。分類、回帰、クラスタリングなど様々な機械学習アルゴリズムを提供します。
- `matplotlib`: データの可視化のためのライブラリ。グラフや図を簡単に描画できます。
- `pandas`: データ操作と分析のためのライブラリ。

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris, load_wine, load_breast_cancer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.utils import shuffle
import seaborn as sns
from tqdm import tqdm
random_state = 42  # 乱数のシード

import warnings
from sklearn.exceptions import ConvergenceWarning

# ConvergenceWarningを無視するように設定
warnings.filterwarnings("ignore", category=ConvergenceWarning)

## データセットの選択

いくつかの有名なデータセットから選択できます。各データセットの概要は以下の通りです：

| データセット名 | 目的 | 特徴量数 | 各特徴量 |
| --- | --- | --- | --- |
| Iris | 3種類のアヤメの品種を分類 | 4 | ガクの長さ・幅、花弁の長さ・幅 |
| Wine | 3種類のワインのタイプを分類 | 13 | アルコール度数、マグネシウム含有量など |
| Breast Cancer | 良性・悪性の腫瘍を分類 | 30 | 半径、テクスチャ、周囲長、面積など |
以下のコードを実行して、使用するデータセットを選択します。

In [ ]:
dataset_name = "Iris" # @param ["Iris","Wine","Breast Cancer"]

from sklearn.datasets import load_iris, load_wine, load_breast_cancer
import pandas as pd
from sklearn.utils import shuffle
import sys
from IPython.display import display

# データセットをロードする関数
def load_dataset(dataset_name):
    if dataset_name == "Iris":
        dataset = load_iris()
        feature_names = dataset.feature_names
        # feature_names = ["ガクの長さ", "ガクの幅", "花弁の長さ", "花弁の幅"]
        target_names = dataset.target_names
    elif dataset_name == "Wine":
        dataset = load_wine()
        feature_names = dataset.feature_names
        target_names = dataset.target_names
    elif dataset_name == "Breast Cancer":
        dataset = load_breast_cancer()
        feature_names = dataset.feature_names
        # target_names = dataset.target_names
        target_names = ["Malignant (bad)", "Benign (good)"]
    else:
        print("誤った番号です")
        return None, None, None, None, None, None

    X = dataset.data
    y = dataset.target
    return dataset.data, dataset.target, pd.DataFrame(X, columns=feature_names), pd.Series(y, name="CLASS"), dataset_name, target_names, feature_names

# データセットのロード
X, y, feature_df, y_df, dataset_name, target_names, feature_names = load_dataset(dataset_name)

if X is not None:
    # クラス名を追加
    y_class_names = y_df.map(lambda x: target_names[x]).rename("CLASS_NAME")

    # 特徴量とクラス、クラス名を結合したDataFrameを作成
    df_combined = pd.concat([y_df, y_class_names, feature_df], axis=1)

    # データをシャッフル
    df_shuffled = shuffle(df_combined, random_state=42).reset_index(drop=True)

    # 表示
    display(df_shuffled.head())

#### **課題 1**
- ひとつのサンプル（花，ワイン，患者）は行ですか，列ですか？
- この表では，各サンプルの
 - クラスはどこですか？
 - １つのサンプルごとに何次元の特徴量がありますか？
- 今回の機械学習モデルでは、各サンプルがもつ ＿＿＿＿＿＿＿ に基づいて、そのデータが各クラスに属する ＿＿＿＿ をそれぞれ計算し、その値が最大となるクラスに分類します。


## データの準備と可視化

### データの構造を理解するための可視化

機械学習の第一歩として、データを理解することが重要です。

まず、データセットの特徴量のうち2つを選んで散布図を描き、データの分布とクラスごとの関係を視覚化します。


In [ ]:
# 任意のクラス数に対応するカラーパレットを生成する関数
def generate_color_palette(n_colors):
    return plt.cm.get_cmap('tab10', n_colors)

# 特徴量のうち、2つを選んで散布図を描く関数
def plot_2d_projection(X, y, feature_indices, feature_names, target_names, title=""):
    plt.figure(figsize=(8, 6))
    palette = generate_color_palette(len(np.unique(y)))
    for target in np.unique(y):
        subset = X[y == target]
        plt.scatter(subset[:, feature_indices[0]], subset[:, feature_indices[1]], label=f"Class {target}: {target_names[target]}", color=palette(target))
    plt.xlabel(f"Feature {feature_indices[0]}: {feature_names[0]}")
    plt.ylabel(f"Feature {feature_indices[1]}: {feature_names[1]}")
    plt.title(title)
    plt.legend()
    plt.show()

# 特徴量の組み合わせを選んでプロット
if X is not None:
    plot_2d_projection(X, y, feature_indices=[0, 1], feature_names=feature_names, target_names=target_names, title=f"{dataset_name} Data - Features 0 and 1")


### データの分割

データセット全体を1つの塊として使用するのではなく、以下の3つの部分に分割します：

1. **教師データ (training data)**:
   モデルの学習に使用します。このデータを使用して、モデルは特徴量と目標変数（ラベル）の関係を学びます。

2. **検証データ (validation data)**:
   モデルの性能を評価し、ハイパーパラメータの調整に使用します。教師データに含まれないデータを使うことで、モデルの汎化性能を確認します。

3. **テストデータ (test data)**:
   最終的なモデルの性能を評価するために使用します。このデータも教師データには含まれていません。テストデータの性能が高ければ、モデルは未知のデータに対しても良好に動作することが期待されます。


In [ ]:
# One-hotエンコーディング
encoder = OneHotEncoder()
y_onehot = encoder.fit_transform(y.reshape(-1, 1)).toarray()

# データを分割 (70% training, 15% validation, 15% test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y_onehot, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 分割結果の確認

# DataFrameの作成
def create_df(X, y, feature_names, target_names):
    y_classes = [target_names[i] for i in y.argmax(axis=1)]
    y_class_series = pd.Series(y_classes, name="CLASS_NAME")
    return pd.concat([pd.DataFrame(X, columns=feature_names), y_class_series], axis=1)

df_train = create_df(X_train, y_train, feature_names, target_names)
df_val = create_df(X_val, y_val, feature_names, target_names)
df_test = create_df(X_test, y_test, feature_names, target_names)

# 表示
print("\n[教師データ]")
print("サイズ:", X_train.shape)
display(df_train.head())

print("\n[検証データ]")
print("サイズ:", X_val.shape)
display(df_val.head())

print("\n[テストデータ]")
print("サイズ:", X_test.shape)
display(df_test.head())

#### **課題２**
すべてのデータを３個に分割しました．それぞれ何に使われますか？

- 教師データ　：
- 検証データ　：
- テストデータ：



## ニューラルネットワークによるモデルの構築とトレーニング

次に、ニューラルネットワークを使用してモデルを構築し、トレーニングを行います。ここでは、scikit-learnの`MLPClassifier`を使用します。

### ニューラルネットワークの概要

ニューラルネットワークは、以下のような主要な要素で構成されます：

1. **入力層 (Input Layer)**:
   特徴量を入力する層です。特徴量の数だけノードがあります。

2. **隠れ層 (Hidden Layer)**:
   入力層と出力層の間にある層です。ニューラルネットワークの学習能力を高めるために使用されます。隠れ層の数や各層のノード数はハイパーパラメータです。

3. **出力層 (Output Layer)**:
   クラスの数だけノードがあります。ここでは、各ノードが特定のクラスに対応します。

### 特徴量の次元数の選択

データをそのまま使用する方法と、plot_2d_projectionで選んだ2次元のみを使用する方法のいずれかで学習を行います。

最初は視覚的な理解のために、2個の特徴量のみを用います。<br>
精度を上げることを目指すには3個以上の特徴量を使うべきですので、`use_two_features = False` に変更してください。その場合は特徴空間や決定境界を可視化できなくなるのでご注意ください。

In [ ]:
# 2次元の特徴量を選択する関数
def select_two_features(X, feature_indices):
    return X[:, feature_indices]

# 特徴量の次元数を選択するかどうか
use_two_features = True
feature_indices = [0, 1]

if use_two_features:
    X_train_transformed = select_two_features(X_train, feature_indices)
    X_val_transformed = select_two_features(X_val, feature_indices)
    X_test_transformed = select_two_features(X_test, feature_indices)
    clms = feature_df.columns[0:2]
else:
    X_train_transformed = X_train
    X_val_transformed = X_val
    X_test_transformed = X_test
    clms = feature_df.columns

### ## 損失関数（Loss Function）の解説

損失関数とは、モデルの予測がどれだけ「**間違っているか**」を測るための**ペナルティ点数**を計算する関数です。学習の目標は、このペナルティ点数をどんどん小さくしていくことです。

ここでは、代表的な3つの損失関数を比較します。

#### **1. マンハッタン距離 (Manhattan Distance / L1損失)** 🏙️

* **考え方**: 予測と正解の「差の絶対値」を合計します。各クラスのズレをそのまま足し合わせる、シンプルで直感的な方法です。
* **特徴**: 外れ値（極端に大きい間違い）の影響を受けにくいという利点があります。
* **計算例**:
    * **理想の出力**: `[0, 1, 0]`
    * **モデルの出力**: `[0.1, 0.7, 0.2]`
    * **損失**: `|0 - 0.1| + |1 - 0.7| + |0 - 0.2| = 0.1 + 0.3 + 0.2 = 0.6`

#### **2. ユークリッド距離 (Euclidean Distance / L2損失)** 📏

* **考え方**: 予測と正解の「差の2乗」を合計します。幾何学的な"直線距離"の2乗を計算するイメージです。
* **特徴**: 差を2乗するため、**大きな間違いに対してより厳しいペナルティ**を与えます。これにより、モデルは大きく外すことを避けるように学習が進みます。
* **計算例**:
    * **理想の出力**: `[0, 1, 0]`
    * **モデルの出力**: `[0.1, 0.7, 0.2]`
    * **損失**: `(0 - 0.1)² + (1 - 0.7)² + (0 - 0.2)² = 0.01 + 0.09 + 0.04 = 0.14`

#### **3. クロスエントロピー (Cross-Entropy / Log Loss)** 🔥

* **考え方**: 2つの「確率分布」がどれだけ似ていないかを測る指標です。分類問題の出力は確率なので、最も理にかなった方法です。
* **特徴**: **「自信満々に、大間違いしたとき」に爆発的に大きなペナルティ**を与えます。これにより、モデルは致命的な間違いから優先的に学習するため、効率的に学習が進みます。分類問題では最も標準的に使われます。
* **計算例**: 正解クラスに対する確率が低いほど、損失は急激に大きくなります (`-log(確率)`)。


In [ ]:
import numpy as np
from sklearn.metrics import log_loss
import torch
import torch.nn.functional as F

#@title 損失関数を選択して実行してください { run: "auto" }
loss_function_name = "クロスエントロピー" #@param ["マンハッタン距離", "ユークリッド距離", "クロスエントロピー"]

# --------------------------------------------------
# 1. 各損失関数の定義 (修正版)
# --------------------------------------------------

def manhattan_loss(y_true_one_hot, y_pred_proba):
    """
    マンハッタン距離 (L1損失) を計算。
    y_true_one_hot は既にone-hot表現であることを想定。
    """
    # y_true_one_hot は既に正しい形なので、そのまま計算に使う
    loss = np.sum(np.abs(y_pred_proba - y_true_one_hot)) / len(y_true_one_hot)
    return loss

def euclidean_loss(y_true_one_hot, y_pred_proba):
    """
    ユークリッド距離の2乗 (MSE) を計算。
    y_true_one_hot は既にone-hot表現であることを想定。
    """
    # y_true_one_hot は既に正しい形なので、そのまま計算に使う
    loss = np.sum((y_pred_proba - y_true_one_hot)**2) / len(y_true_one_hot)
    return loss

def cross_entropy_loss(y_true_one_hot, y_pred_proba):
    """
    クロスエントロピー損失 (log_loss) を計算。
    scikit-learnのlog_lossはクラス番号を要求するため、one-hotを元に戻す。
    """
    # one-hot表現 (例: [0, 1, 0]) からクラス番号 (例: 1) に変換
    y_true_labels = np.argmax(y_true_one_hot, axis=1)
    return log_loss(y_true_labels, y_pred_proba)

# --------------------------------------------------
# 2. 「関数ポインタ」の役割を果たす辞書を作成
# --------------------------------------------------
loss_functions = {
    "マンハッタン距離": manhattan_loss,
    "ユークリッド距離": euclidean_loss,
    "クロスエントロピー": cross_entropy_loss,
}

# プルダウンメニューで選択された名前から、使用する関数を取得
selected_loss_func = loss_functions[loss_function_name]

### モデルのトレーニング

まずモデルを定義します。

隠れ層（中間層）の数やノード数を調節してください。<br>
複雑にすればするほど難しい問題が解けるようになる反面、教師データが少ない場合に過学習が起きやすくなったり、計算時間が増えたりします。

In [ ]:
hidden_layer_sizes = [5, ]  # 隠れ層のノード数。複数の層にしたい場合は [3,4] や [3,5,4] など
batch_size = 32            # バッチサイズ（ミニバッチサイズ）。１回の学習に用いられるサンプルの数

import matplotlib.pyplot as plt
import networkx as nx

def draw_mlp_network(input_nodes, hidden_layers, output_nodes):
    """
    MLPのネットワークを描画する関数

    Parameters:
    - input_nodes: 入力層のノード数
    - hidden_layers: 隠れ層のノード数のリスト
    - output_nodes: 出力層のノード数
    """
    G = nx.DiGraph()

    # ノードの追加
    layers = [input_nodes] + hidden_layers + [output_nodes]
    layer_pos = []
    max_nodes = max(layers)

    for i, layer in enumerate(layers):
        # 中央揃えにするためのオフセット計算
        offset = (max_nodes - layer) / 2
        current_layer = []
        for j in range(layer):
            if i == 0:
                node_id = f'in_{j}'
            elif i == len(layers) - 1:
                node_id = f'out_{j}'
            else:
                node_id = f'hidden{i-1}_{j}'
            G.add_node(node_id, pos=(i, max_nodes - (j + offset)))
            current_layer.append(node_id)
        layer_pos.append(current_layer)

    # エッジの追加
    for i in range(len(layers) - 1):
        for node_start in layer_pos[i]:
            for node_end in layer_pos[i + 1]:
                G.add_edge(node_start, node_end)

    # ノードの位置を取得
    pos = nx.get_node_attributes(G, 'pos')

    # グラフの描画
    plt.figure(figsize=(8, 6))  # 横幅を800pxに設定するための調整
    nx.draw(G, pos, with_labels=True, node_size=1500, node_color="lightblue", font_size=8)
    plt.title('MLP Network Structure')
    plt.show()


# モデルの構築
mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, max_iter=1, warm_start=True, random_state=random_state, batch_size=batch_size)

# 構造の描画
draw_mlp_network(input_nodes=X_train_transformed.shape[1], hidden_layers=hidden_layer_sizes, output_nodes=len(target_names))

モデルを定義できたら学習を行いましょう。
最大エポック数を調節してください。

**ネットワーク構造を書き換えた場合、上のセルを必ず実行して、反映させてから学習してください。**

In [ ]:
max_epochs = 1000           # 最大エポック数

# トレーニングループ
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

# tqdmを使って進行状況バーを表示
with tqdm(total=max_epochs, leave=False) as pbar:
    for epoch in range(max_epochs):
        mlp.fit(X_train_transformed, y_train)

        # 教師データでの予測と評価
        y_train_pred = mlp.predict(X_train_transformed)
        y_train_pred_proba = mlp.predict_proba(X_train_transformed)
        train_accuracy = accuracy_score(y_train, y_train_pred)
        train_loss = selected_loss_func(y_train, y_train_pred_proba)

        # 検証データでの予測と評価
        y_val_pred = mlp.predict(X_val_transformed)
        y_val_pred_proba = mlp.predict_proba(X_val_transformed)
        val_accuracy = accuracy_score(y_val, y_val_pred)
        val_loss = selected_loss_func(y_val, y_val_pred_proba)

        # 記録
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)

        # tqdmの説明文を更新
        pbar.set_description(f"Epoch {epoch + 1}/{max_epochs} - Train loss: {train_loss:.4f}, Val loss: {val_loss:.4f}, Train acc: {train_accuracy:.4f}, Val acc: {val_accuracy:.4f}")
        pbar.update(1)

# 最後のエポックの結果を表示
print(f"Epoch {max_epochs}/{max_epochs} - Train accuracy: {train_accuracy:.4f}, Train loss: {train_loss:.4f}, Val accuracy: {val_accuracy:.4f}, Val loss: {val_loss:.4f}")

# プロット用の間引き
def reduce_points(data, num_points):
    if len(data) > num_points:
        indices = np.linspace(0, len(data) - 1, num_points, dtype=int)
        return [data[i] for i in indices]
    return data

# グラフで学習の様子を表示

num_points = 100
epochs = range(1, max_epochs + 1)
epochs_reduced = reduce_points(list(epochs), num_points)
train_accuracies_reduced = reduce_points(train_accuracies, num_points)
val_accuracies_reduced = reduce_points(val_accuracies, num_points)
train_losses_reduced = reduce_points(train_losses, num_points)
val_losses_reduced = reduce_points(val_losses, num_points)

# プロット
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs_reduced, train_accuracies_reduced, label='Train Accuracy')
plt.plot(epochs_reduced, val_accuracies_reduced, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.legend()
plt.title('Accuracy per Epoch')

plt.subplot(1, 2, 2)
plt.plot(epochs_reduced, train_losses_reduced, label='Train Loss')
plt.plot(epochs_reduced, val_losses_reduced, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim(0, max(max(train_losses_reduced), max(val_losses_reduced)))
plt.legend()
plt.title('Loss per Epoch')

plt.show()

#### **課題３**

- 学習では ＿＿＿＿＿＿ の値を小さくすることを目指します．
 - ある入力サンプルから得られた仮の出力（各クラスに属する＿＿＿＿）と，正解の ＿＿＿＿＿＿ 表現を比較して，その値が少しだけ小さくなるようにネットワークを更新します．この比較に用いる関数は ＿＿＿＿＿関数とよばれます．

- 左のグラフは＿＿＿＿＿＿，右のグラフは＿＿＿＿＿＿です．
 - 学習で小さくしようとしているのは右ですか，左ですか？

- 今回は最大エポック数を指定して学習を繰り返しましたが、エポック数よりもずっと多くの回数の学習が行われます。

 - 学習を繰り返すのち、＿＿＿＿＿ をひととおり使い切ることを 1 エポックと数える。

 - データ量が多すぎて一度に使い切れないので、少数のデータを取り出しては学習を行う。取り出す単位を ＿＿＿＿＿＿ と呼ぶ。これを略してバッチといい、１個に含まれる個数をプログラム中では `batch_size`（バッチサイズ）としている。今回のバッチサイズは ＿＿＿＿＿ である。

 - 学習のたび、＿＿＿＿＿関数を用いて ＿＿＿＿＿＿＿ と ＿＿＿＿＿＿ を比較し、その関数が小さくなるようにネットワークを修正する。その修正の処理を ＿＿＿＿＿ という[（参考）](https://hogetech.info/ml/dl/backpropagation)。


### 教師データや検証データで試す

In [ ]:
# 教師データでの予測
y_train_pred = mlp.predict(X_train_transformed)
y_train_pred_proba = mlp.predict_proba(X_train_transformed)

# 教師データでの評価
train_accuracy = accuracy_score(y_train, y_train_pred)
train_loss = selected_loss_func(y_train, y_train_pred_proba)

print(f"教師データでの精度: {train_accuracy:.3f}")
print(f"教師データでの損失: {train_loss:.3f}")

# 検証データでの予測
y_val_pred = mlp.predict(X_val_transformed)
y_val_pred_proba = mlp.predict_proba(X_val_transformed)

# 検証データでの評価
val_accuracy = accuracy_score(y_val, y_val_pred)
val_loss = selected_loss_func(y_val, y_val_pred_proba)

print(f"検証データでの精度: {val_accuracy:.3f}")
print(f"検証データでの損失: {val_loss:.3f}")

### 検証データを１個入力して試す

作成されたネットワークがどのように動作するのか、検証データを１個用いて確認してみます。

- **ネットワークへの入力（特徴量）**

    検証データから特定のサンプルを選び、その特徴量ベクトルを表示します。これは、ネットワーク（モデル）に入力されるデータで、各特徴量が数値として表現されています。

- **ネットワークの出力（確率）**

    選択した入力に対して、モデルが各クラスに属する確率を計算します。この確率は、入力がそれぞれのクラスに分類される可能性を示しており、各クラスに対して1つの確率値が出力されます。

- **推定されたクラス番号の one-hot 表現**

    モデルが予測したクラスのone-hot表現を表示します。one-hot表現とは、正解クラスを1、それ以外のクラスを0とするベクトル形式のことです。例えば4クラスの場合、[0, 0, 1, 0]はクラス 2 を示します（注：クラス番号は 0 から始まります）。

- **クラス推定の表示（整数表現）**

    one-hot表現のベクトルを整数に変換し、予測されたクラスを数値として示します。

In [ ]:
# 特徴量ベクトルを入力し、各クラスの確率を出力
sample_index = 1  # 任意のサンプルインデックス
sample_input = X_val_transformed[sample_index].reshape(1, -1)  # 1サンプルの入力データ
print("ネットワークへの入力（特徴量）　　 ：", sample_input)

# 確率の出力
class_probabilities = mlp.predict_proba(sample_input)
print("ネットワークの出力（確率）　　 　　：", class_probabilities)

# クラス推定
predicted_class = mlp.predict(sample_input)
print("推定されたクラス番号の one-hot 表現：", predicted_class)
print("推定されたクラス番号　　　　　　　 ：", np.argmax(predicted_class))

# クラス推定
predicted_class = mlp.predict(sample_input)
print("正解の one-hot 表現　　　　　　　　：", y_train[sample_index])
print("正解　　　　　　　　　　　　　　　 ：", np.argmax(y_train[sample_index]))

#### **課題４**

- 出力を読み取りながら埋めてください：

  あるサンプルの ＿＿＿＿＿＿＿ をネットワークに入力したところ，＿＿＿＿＿＿ が出力された．

  出力は，そのサンプルが各クラスに属する確率を表すものである．その中では ＿＿＿＿ 番目のクラスが最大となったことから，推定されたクラスは ＿＿＿＿ となった．

### テストデータでの評価

モデルの精度評価としてテストデータを使用します。<br>
学習中に精度を確認するのに用いた検証データとは異なり、学習中にはテストデータはまったく使用されていません。

In [ ]:
# 全テストデータの結果を表示
all_test_samples = []

for i in range(len(X_test_transformed)):
    test_sample = X_test_transformed[i].reshape(1, -1)
    probabilities = mlp.predict_proba(test_sample)[0]
    predicted_class_index = np.argmax(probabilities)
    predicted_class_name = target_names[predicted_class_index]
    true_class_index = np.argmax(y_test[i])
    true_class_name = target_names[true_class_index]

    test_sample_df = pd.DataFrame(test_sample, columns=clms)
    test_sample_df["正解"] = f"クラス{true_class_index}-{true_class_name}"
    test_sample_df["推論"] = f"クラス{predicted_class_index}-{predicted_class_name}"

    # 各クラスの確率を追加
    for class_index, prob in enumerate(probabilities):
        test_sample_df[f"クラス{class_index}の確率"] = round(prob, 3)

    # カラムの順番を調整
    probability_columns = [f"クラス{class_index}の確率" for class_index in range(len(probabilities))]
    test_sample_df = test_sample_df[
        ["正解", "推論"] + probability_columns + clms.tolist()
    ]

    all_test_samples.append(test_sample_df)

all_test_samples_df = pd.concat(all_test_samples, ignore_index=True)

print("全テストデータの結果:")
display(all_test_samples_df)


さらに、テストデータにおいて精度（正解率）および損失を計算してみましょう。

In [ ]:
# テストデータでの予測
y_test_pred = mlp.predict(X_test_transformed)
y_test_pred_proba = mlp.predict_proba(X_test_transformed)

# テストデータでの評価
test_accuracy = accuracy_score(y_test, y_test_pred)
test_loss = selected_loss_func(y_test, y_test_pred_proba)

print(f"テストデータでの精度: {test_accuracy:.4f}")
print(f"テストデータでの損失: {test_loss:.4f}")

#### **課題５**
- 「全テストデータの結果」の表を読み取ってください
 - たとえば最初の行のサンプルは、（各クラスの確率を写してください）と出力された。最大の確率をもつクラスが ＿＿＿ であったため、「推論」の結果も ＿＿＿ となり、（正解に一致していた／正解と異なっていた）。

### 2次元プロットによるモデルの可視化

2個（2次元）の特徴量のみで学習を行った場合、得られたモデルの分類領域を2次元で視覚化できます。<br>
テストデータに対して、その点に対するモデルの予測結果をプロットします。

In [ ]:
# グリッドサーチによる領域プロット
def plot_decision_boundary(clf, X, y, feature_indices, feature_names, title=""):
    # グリッドを生成
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                         np.arange(y_min, y_max, 0.02))

    # グリッド全体で予測を実行
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = np.argmax(Z, axis=1)
    Z = Z.reshape(xx.shape)

    # プロットの描画
    plt.figure(figsize=(8, 6))
    plt.contourf(xx, yy, Z, alpha=0.3)
    palette = plt.cm.get_cmap('tab10', len(np.unique(np.argmax(y, axis=1))))
    for target in np.unique(np.argmax(y, axis=1)):
        subset = X[np.argmax(y, axis=1) == target]
        plt.scatter(subset[:, feature_indices[0]], subset[:, feature_indices[1]], label=f"Class {target}: {target_names[target]}", color=palette(target))
    plt.xlabel(f"Feature {feature_indices[0]}: {feature_names[0]}")
    plt.ylabel(f"Feature {feature_indices[1]}: {feature_names[1]}")
    plt.title(title)
    plt.legend()
    plt.show()

# 特徴量の組み合わせを選んでプロット
if use_two_features == True:
  plot_decision_boundary(mlp, X_test_transformed, y_test, feature_indices, feature_names, title=f"Decision Boundary for {dataset_name} Dataset")
else:
  print("3次元以上の特徴量を使用しているため実行されません")

#### **課題６**

- クラスを分ける境界のことを ＿＿＿＿＿＿＿＿ という．
 - これを適切な位置に決めることが学習であるともいえる．
 - 各クラスが概ね良好に分離できていれば学習は成功
 - あまりにも細かくクネクネと分離している状況は ＿＿＿＿＿ とされる．教師データでは高い精度を得られるが，テストデータにはむしろ精度が下がってしまう．学習のしすぎを防ぐために，＿＿＿データを用いてハイパーパラメータを調節したり，ちょうどよいタイミングでで学習を停止したりする工夫が行われる．

#### **課題７**

学んだことをもとに，ご自身の言葉でまとめてください．

- ニューラルネットワークの入力と出力
- 分類問題における学習とは、何を用いて何をするもの？
 - クラス・サンプル・特徴量・決定境界・損失・one-hot といった言葉を用いてください
- 学習されたモデルをどうやって使うと，新たなサンプルのクラスを推定できる？



## 発展

今回，１サンプルの特徴量は，多くても 30 次元でした．

しかし，たとえば画像を判別するなら，とても小さな画像でも 縦 32 × 横 32 × RGB 3チャンネル ＝ 3072 次元くらいはあります．この場合，画像を特徴量に変換（**特徴抽出**）した上で，判別に使うことになります．

この特徴抽出をネットワーク内部で行ってくれるものとして有名なのが **畳み込み層**（Convolutional Layer）です．また，それを含むニューラルネットワークが畳み込みニューラルネットワーク（Convolutional Neural Network; **CNN**）です．

- 多次元配列の扱いについては [NumPy の演習](https://github.com/shizoda/education/blob/main/machine_learning/numpy.ipynb)

- デジタル画像の表現については [画像表現の演習](https://github.com/shizoda/education/blob/main/image/colors.ipynb)

- 畳み込みやフィルタ処理については [畳み込みの演習](https://github.com/shizoda/education/blob/main/image/conv.ipynb)

これらをご存知の方はこのまま

- [CNN を用いた画像分類の演習](https://github.com/shizoda/education/blob/main/machine_learning/cnn/cifar10_pytorch.ipynb)

に進んでください．